In [1]:
from llama_index.callbacks import CallbackManager, LlamaDebugHandler

In [2]:
llama_debug_handler = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug_handler])

In [3]:
import torch

from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
max_memory = {"0": "10GiB", "cpu": "5GiB"}

model_id = "mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized(model_id, 
                                           #device=device,
                                           #max_memory=max_memory,
                                           device_map="auto",
                                           use_safetensors=True,
                                           use_fast_inference=True,)

skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [27]:
pipe = pipeline("translation_xx_to_yy", 
                model=model, 
                tokenizer=tokenizer,
                max_length=5120,
                #max_new_tokens=4096,
                )

The model 'LlamaGPTQForCausalLM' is not supported for translation_xx_to_yy. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [28]:
llm = HuggingFacePipeline(pipeline=pipe)

In [7]:
#from langchain.embeddings import HuggingFaceEmbeddings
#from llama_index import LangchainEmbedding
from llama_index.embeddings import HuggingFaceEmbedding

In [8]:
model_id = "sentence-transformers/all-MiniLM-l6-v2"
embed_model = HuggingFaceEmbedding(model_name=model_id, max_length=512, device=device)
#embed_model = LangchainEmbedding()

In [9]:
import nest_asyncio

from llama_index import ServiceContext, StorageContext, get_response_synthesizer
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index.vector_stores import SimpleVectorStore
from llama_index.indices.document_summary import DocumentSummaryIndex

# 非同期処理の有効化
nest_asyncio.apply()

In [29]:
node_parser = SentenceWindowNodeParser(window_size=3, 
                                  window_metadata_key="sentence_window", 
                                  original_text_metadata_key="original_text"
                                  )

storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore(),
    vector_store=SimpleVectorStore(),
    index_store=SimpleIndexStore(),
)

service_context = ServiceContext.from_defaults(
    llm = llm, 
    embed_model=embed_model, 
    node_parser=node_parser, 
    callback_manager=callback_manager,
    chunk_size=514,
    chunk_overlap=20,
    )

In [11]:
from src.XMLUtils import DocumentCreator

In [12]:
base_path = "/home/paper_translator/data"
document_name = "Ask_more_know_better_Reinforce-Learned_Prompt_Questions_for_Decision_Making_with_Large_Language_Models"
document_path = f"{base_path}/documents/{document_name}"
xml_path = f"{document_path}/{document_name}.tei.xml"

creator = DocumentCreator()
creator.load_xml(xml_path, contain_abst=False)
documents = creator.create_docs()

In [13]:
from llama_index.llms.base import ChatMessage, MessageRole
from llama_index.prompts import ChatPromptTemplate

In [14]:
# QAシステムプロンプト
TEXT_QA_SYSTEM_PROMPT = ChatMessage(
    content=(
                "#依頼\n"
                "あなたは高度な理解能力を持ち、複雑なテキストも簡潔に要約することができるAIです。\n"
                "事前知識ではなく、提供されたコンテキストに基づいて精確な回答を行ってください。\n"
                "#従うべきルール\n"
                "1. 略語や初出の用語には解説を加え、AI分野やコンピュータの初心者も理解できるように工夫してください。\n"
                "2. 回答内で指定されたコンテキストを直接参照しないでください。\n"
                "3. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。\n"
                "4. 出力は日本語で行ってください。"
                "#手順\n"
                "1. 与えられたコンテキストに含まれる主要なポイントやコンセプトを細かく分解してください。\n"
                "2. それぞれのポイントやコンセプトに対して詳細な説明を加えてください。\n"
                "3. まずは指示に従って、文書の初版を作成してください。\n"
                "4. 作成した初版をルールに従っているか自己分析してください。\n"
                "5. 自己分析の結果を踏まえて、文書を改善してください。\n"
    ),
    role=MessageRole.SYSTEM,
)

# QAプロンプトテンプレートメッセージ
TEXT_QA_PROMPT_TMPL_MSGS = [
            TEXT_QA_SYSTEM_PROMPT,
            ChatMessage(
                content=(
                    "複数のソースからのコンテキスト情報を以下に示します。\n"
                    "---------------------\n"
                    "{context_str}\n"
                    "---------------------\n"
                    "予備知識ではなく、複数のソースからの情報を考慮して質問に答えてください。\n"
                    "疑問がある場合は、「情報無し」と答えてください。\n"
                    "Query: {query_str}\n"
                    "Answer: "
                ),
                role=MessageRole.USER,
            ),
]

# チャットQAプロンプト
CHAT_TEXT_QA_PROMPT = ChatPromptTemplate(
            message_templates=TEXT_QA_PROMPT_TMPL_MSGS
)

In [15]:
# QAシステムプロンプト
TEXT_QA_SYSTEM_PROMPT = ChatMessage(
    content=(
                "#依頼\n"
                "あなたは高度な理解能力を持ち、複雑なテキストも簡潔に要約することができるAIです。\n"
                "事前知識ではなく、提供されたコンテキストに基づいて精確な回答を行ってください。\n"
                "#従うべきルール\n"
                "1. 略語や初出の用語には解説を加え、AI分野やコンピュータの初心者も理解できるように工夫してください。\n"
                "2. 回答内で指定されたコンテキストを直接参照しないでください。\n"
                "3. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。\n"
                "4. 出力は日本語で行ってください。"
                "#手順\n"
                "1. 与えられたコンテキストに含まれる主要なポイントやコンセプトを細かく分解してください。\n"
                "2. それぞれのポイントやコンセプトに対して詳細な説明を加えてください。\n"
                "3. まずは指示に従って、文書の初版を作成してください。\n"
                "4. 作成した初版をルールに従っているか自己分析してください。\n"
                "5. 自己分析の結果を踏まえて、文書を改善してください。\n"
    ),
    role=MessageRole.SYSTEM,
)

# ツリー要約プロンプトメッセージ
TREE_SUMMARIZE_PROMPT_TMPL_MSGS = [
            TEXT_QA_SYSTEM_PROMPT,
            ChatMessage(
                content=(
                    "複数のソースからのコンテキスト情報を以下に示します。\n"
                    "---------------------\n"
                    "{context_str}\n"
                    "---------------------\n"
                    "予備知識ではなく、複数のソースからの情報を考慮して質問に答えてください。\n"
                    "疑問がある場合は、「情報無し」と答えてください。\n"
                    "Query: {query_str}\n"
                    "Answer: "
                ),
                role=MessageRole.USER,
            ),
]

# ツリー要約プロンプト
CHAT_TREE_SUMMARIZE_PROMPT = ChatPromptTemplate(
            message_templates=TREE_SUMMARIZE_PROMPT_TMPL_MSGS
)

In [30]:
response_synthesizer = get_response_synthesizer(
            service_context=service_context,
            text_qa_template=CHAT_TEXT_QA_PROMPT,  # QAプロンプト
            summary_template=CHAT_TREE_SUMMARIZE_PROMPT,  # TreeSummarizeプロンプト
            response_mode="tree_summarize",
            callback_manager=callback_manager,
            use_async=True,
        )

In [31]:
doc_summary_index = DocumentSummaryIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
    summary_query="提供されたテキストの内容を要約してください。",
)

current doc id: 0
**********
Trace: index_construction
    |_CBEventType.SYNTHESIZE ->  13.234265 seconds
      |_CBEventType.TEMPLATING ->  4.4e-05 seconds
      |_CBEventType.LLM ->  0.0 seconds
      |_CBEventType.LLM ->  0.0 seconds
      |_CBEventType.TEMPLATING ->  4.3e-05 seconds
      |_CBEventType.LLM ->  0.0 seconds
      |_CBEventType.LLM ->  0.0 seconds
      |_CBEventType.TEMPLATING ->  4.3e-05 seconds
      |_CBEventType.LLM ->  0.0 seconds
      |_CBEventType.LLM ->  0.0 seconds
      |_CBEventType.EXCEPTION ->  0.0 seconds
      |_CBEventType.EXCEPTION ->  0.0 seconds
        |_CBEventType.EXCEPTION ->  0.0 seconds
**********


Task exception was never retrieved
future: <Task finished name='Task-1' coro=<run_async_tasks.<locals>._gather() done, defined at /home/paper_translator/.venv/lib/python3.11/site-packages/llama_index/async_utils.py:36> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/home/paper_translator/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_510/2671373537.py", line 1, in <module>
    doc_summary_index = DocumentSummaryIndex.from_documents(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/paper_translator/.venv/lib/python3.11/site-packages/llama_index/indices/base.py", line 102, in from_documents
    return cls(
           ^^^^
  File "/home/paper_translator/.venv/lib/python3.11/site-packages/llama_index/indices/document_summary/base.py", line 77, in __init__
    super().__init__(
  File "/home/paper_translator/.venv/lib

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/paper_translator/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_510/2671373537.py", line 1, in <module>
    doc_summary_index = DocumentSummaryIndex.from_documents(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/paper_translator/.venv/lib/python3.11/site-packages/llama_index/indices/base.py", line 102, in from_documents
    return cls(
           ^^^^
  File "/home/paper_translator/.venv/lib/python3.11/site-packages/llama_index/indices/document_summary/base.py", line 77, in __init__
    super().__init__(
  File "/home/paper_translator/.venv/lib/python3.11/site-packages/llama_index/indices/base.py", line 71, in __init__
    index_struct = self.build_index_from_nodes(nodes)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/paper_translator/.venv/lib/python3.11/site-packages